In [1]:
import snap
import numpy as np
import operator

In [12]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

def union(lst1, lst2):
    return list(set(lst1) | set(lst2))

def Jaccard(x, y):
    neighbor_x = []
    neighbor_y = []
    degx = x.GetDeg()
    degy = y.GetDeg()
    for i in range(degx):
        neighbor_x.append(x.GetNbrNId(i))
    for i in range(degy):
        neighbor_y.append(y.GetNbrNId(i))
    return float(len(intersection(neighbor_x, neighbor_y))) / len(union(neighbor_x, neighbor_y))

def preferential_attachment(x,y,n):
    neighbor_x = []
    neighbor_y = []
    degx = x.GetDeg()
    degy = y.GetDeg()
#     for i in range(degx):
#         neighbor_x.append(x.GetNbrNId(i))
#     for i in range(degy):
#         neighbor_y.append(y.GetNbrNId(i))
    return float(degx*degy) / (2*n)

def readFile(filename):
    deleted = []
    with open('deleted_edges.txt', 'r') as f:
        for line in f:
            nodes = line.split()
            deleted.append((int(nodes[0]), int(nodes[1])))
    return deleted

def predict_preferential():
    scores = {}
    deleted = readFile('deleted_edges.txt')
    FIn = snap.TFIn("train.graph")
    train = snap.TUNGraph.Load(FIn)
    maxDisease = 600608

    # loop over disease nodes
    for x in train.Nodes():
        nidx = x.GetId()
        if nidx <= maxDisease:
            # loop over chemical nodes
            for y in train.Nodes():
                nidy = y.GetId()
                 # only predict link between chemical and disease
                if nidy > maxDisease and not train.IsEdge(nidx, nidy):
                    scores[(nidx,nidy)] = preferential_attachment(x,y,train.GetEdges())

    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)        
    correct = 0
    n = 2000
    FIn = snap.TFIn("test.graph")
    test = snap.TUNGraph.Load(FIn)
    
    for prediction in sorted_scores[:n]:
        if test.IsEdge(prediction[0][0], prediction[0][1]) :
            correct += 1

    print float(correct)/n
    
predict_preferential()
# print readFile('deleted_edges.txt')

0.0345


In [ ]:
def predict_distance():
    scores = {}
    deleted = readFile('deleted_edges.txt')
    FIn = snap.TFIn("train.graph")
    train = snap.TUNGraph.Load(FIn)
    maxDisease = 600608

    # loop over disease nodes
    for x in train.Nodes():
        nidx = x.GetId()
        if nidx <= maxDisease:
            # loop over chemical nodes
            for y in train.Nodes():
                nidy = y.GetId()
                 # only predict link between chemical and disease
                if nidy > maxDisease and not train.IsEdge(nidx, nidy):
                    scores[(nidx,nidy)] = snap.GetShortPath(train,nidx,nidy)

    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1))        
    correct = 0
    n = 2000
    FIn = snap.TFIn("test.graph")
    test = snap.TUNGraph.Load(FIn)
    
    for prediction in sorted_scores[:n]:
        if test.IsEdge(prediction[0][0], prediction[0][1]) :
            correct += 1

    print float(correct)/n
    
predict_distance()